In [1]:
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd
import json
import selenium
from selenium import webdriver
import os
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time 
import numpy as np
#Importo todo lo necesario para el scraping

In [2]:
#driver configuration
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
opciones.add_argument('--incognito')
opciones.add_argument("--no-sandbox")
opciones.add_argument("--disable-setuid-sandbox")

In [3]:
driver = "./chromedriver.exe" #ejecuto el driver
driver = webdriver.Chrome(driver,options = opciones)
url = 'https://www.worlddata.info/life-expectancy.php' #selecciono la web sobre la que trabajare
driver.get(url) #accedo a la info de la web 
time.sleep(5)
boton = driver.find_element_by_css_selector("#cookieconsent_box > button.cc_accept.cc_right")
boton.click() #hago click en las cookis que me aparecen para acceder a los datos

/tmp/ipykernel_7855/3838757253.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver,options = opciones)
/tmp/ipykernel_7855/3838757253.py:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  boton = driver.find_element_by_css_selector("#cookieconsent_box > button.cc_accept.cc_right")


In [4]:
html=req.get(url)
soup = bs(html.text,'html.parser')
tabla = soup.find_all('table',class_="std100 hover") 
type(tabla) #obtengo un objeto de bs en el cual se encuentra la información que busco

bs4.element.ResultSet

In [5]:
link=tabla[0].find_all('td')


In [6]:
link[0].text.strip().split('\n')[0]
link[1].text.strip().split('\n')[0]
len(link)


600

In [7]:
arr = []
for b in range(0,600):
    arr.append(link[b].text.strip().split('\n')[0]) #obtengo un array con todos mis datos juntos

In [8]:
len(arr)

600

In [10]:
def marico(arr, size):
     arrs = []
     while len(arr) > size:
         pice = arr[:size]
         arrs.append(pice)
         arr   = arr[size:]
     arrs.append(arr)
     return arrs

In [11]:
rows = marico(arr, 5) #sobre mis rows extraidas del objeto bs aplico mi funcion y separo mis datos para trabajar más facil 

In [12]:
rows

[['Hong Kong', '82.2 years', '88.1 years', '7.0 ‰', '6.5 ‰'],
 ['Switzerland', '81.9 years', '85.6 years', '10.0 ‰', '7.9 ‰'],
 ['Japan', '81.4 years', '87.5 years', '7.0 ‰', '11.1 ‰'],
 ['Singapore', '81.4 years', '85.7 years', '8.8 ‰', '5.0 ‰'],
 ['Sweden', '81.3 years', '84.7 years', '11.1 ‰', '8.6 ‰'],
 ['Macao', '81.3 years', '87.2 years', '10.9 ‰', '4.0 ‰'],
 ['Norway', '81.2 years', '84.7 years', '10.2 ‰', '7.6 ‰'],
 ['Italy', '81.1 years', '85.4 years', '7.0 ‰', '10.5 ‰'],
 ['Israel', '81.0 years', '84.7 years', '20.1 ‰', '5.1 ‰'],
 ['Iceland', '81.0 years', '84.2 years', '12.3 ‰', '6.3 ‰'],
 ['Spain', '80.9 years', '86.2 years', '7.6 ‰', '8.8 ‰'],
 ['Australia', '80.9 years', '85.0 years', '12.1 ‰', '6.7 ‰'],
 ['Netherlands', '80.5 years', '83.6 years', '9.7 ‰', '8.8 ‰'],
 ['Ireland', '80.4 years', '84.3 years', '12.1 ‰', '6.3 ‰'],
 ['Luxembourg', '80.3 years', '84.7 years', '10.0 ‰', '6.9 ‰'],
 ['South Korea', '80.3 years', '86.3 years', '5.9 ‰', '5.7 ‰'],
 ['Canada', '80.0 y

In [13]:
df= pd.DataFrame(rows) #paso a un dataframe la info que he obtenido y acabo de limpiar 

In [14]:
df.columns = ['country','life_expectancy_males','life_expectancy_females','birth_rate%','death_rate%'] 
#doy un nuevo nombre a mis columnas

In [15]:
df

,country,life_expectancy_males,life_expectancy_females,birth_rate%,death_rate%
0,Hong Kong,82.2 years,88.1 years,7.0 ‰,6.5 ‰
1,Switzerland,81.9 years,85.6 years,10.0 ‰,7.9 ‰
2,Japan,81.4 years,87.5 years,7.0 ‰,11.1 ‰
3,Singapore,81.4 years,85.7 years,8.8 ‰,5.0 ‰
4,Sweden,81.3 years,84.7 years,11.1 ‰,8.6 ‰
...,...,...,...,...,...
115,Ivory Coast,56.6 years,59.1 years,35.5 ‰,9.9 ‰
116,Somalia,55.7 years,59.1 years,41.6 ‰,10.7 ‰
117,Nigeria,53.8 years,55.6 years,37.4 ‰,11.6 ‰
118,Chad,52.8 years,55.7 years,41.7 ‰,11.9 ‰


In [16]:
def quitamierda(string):
    '''
    Funcion que limpia de % mis columnas para evitar problemas
    ignora los NaN para poder ejecutarse
    '''
    if type(string)==float:
        return string
    else:
        return string.replace('‰',' ').strip()

In [17]:
quitamierda('37.4 ‰')

'37.4'

In [18]:
df['birth_rate%'].isna().sum()

0

In [19]:
df['birth_rate%'] = df['birth_rate%'].apply(quitamierda)
df['death_rate%'] = df['death_rate%'].apply(quitamierda)

In [20]:
df

,country,life_expectancy_males,life_expectancy_females,birth_rate%,death_rate%
0,Hong Kong,82.2 years,88.1 years,7.0,6.5
1,Switzerland,81.9 years,85.6 years,10.0,7.9
2,Japan,81.4 years,87.5 years,7.0,11.1
3,Singapore,81.4 years,85.7 years,8.8,5.0
4,Sweden,81.3 years,84.7 years,11.1,8.6
...,...,...,...,...,...
115,Ivory Coast,56.6 years,59.1 years,35.5,9.9
116,Somalia,55.7 years,59.1 years,41.6,10.7
117,Nigeria,53.8 years,55.6 years,37.4,11.6
118,Chad,52.8 years,55.7 years,41.7,11.9


In [21]:
df.to_csv('life_expectancy_data.csv', index=False) #guardo mi dataframe limpio en csv